In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import mutagen
import os
import pickle
import random
import wave
import numpy as np
from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

import utils
import classifier
import multi_classifier as mclass
import irmasTrainUtils as trainUtils
import irmasTestUtils as testUtils

In [3]:
df_train = utils.load_pickled_dataset("../train.pkl")
df_valid = utils.load_pickled_dataset("../validation.pkl")
df_train = df_train.sample(frac=1)
df_valid = df_valid.sample(frac=1)

In [4]:
model = classifier.make_model()
total_history = []

In [20]:
history = classifier.train_model(model, df_train, batch_size=32, epochs=5)
total_history.extend(history.history['loss'])

Epoch 1/5
377/377 [==============================] - 78s 205ms/step - loss: 1.1063
Epoch 2/5
377/377 [==============================] - 71s 189ms/step - loss: 1.1098
Epoch 3/5
377/377 [==============================] - 73s 194ms/step - loss: 1.1076
Epoch 4/5
377/377 [==============================] - 70s 185ms/step - loss: 1.0819
Epoch 5/5
377/377 [==============================] - 65s 173ms/step - loss: 1.0775


In [18]:
model.save_weights("weights")

In [19]:
pickle.dump( total_history, open( "model_history", "wb" ) )

In [6]:
test = classifier.make_model()
test.load_weights("weights")
print(classifier.get_accuracy(test, df_train))
print(classifier.get_accuracy(test, df_valid))

0.7036668325866932
0.6047197640117994


In [7]:
smaller = mclass.make_model()
for layer in test.layers:
    for i in range(len(smaller)):
        for v in range(len(smaller[i].layers)):
            if smaller[i].layers[v].name == layer.name:
                smaller[i].layers[v].set_weights(layer.get_weights())